In [2]:
# from datasets import load_dataset

# dataset = load_dataset("amazon_polarity")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import pickle
# with open('amazon.pkl', 'wb') as file:
#     pickle.dump(dataset, file)

In [4]:
with open('amazon.pkl', 'rb') as file:
    dataset = pickle.load(file)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})


In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

# Convert train and test datasets to arrays
train_data_title = train_dataset['title']
train_data_content = train_dataset['content']
train_labels = train_dataset['label']
test_data_title = test_dataset['title']
test_data_content = test_dataset['content']
test_labels = test_dataset['label']



In [7]:
print(train_data_title[0])
print(train_data_content[0])
print(train_labels[0])


Stuning even for the non-gamer
This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
1


In [8]:
# train_data = train_data_title + train_data_content
# test_data = test_data_title + test_data_content

train_data = [None] * len(train_data_title)
for i in range(len(train_data_title)):
    train_data[i] = train_data_title[i] + "mid" + train_data_content[i]

test_data = [None] * len(test_data_title)
for i in range(len(test_data_title)):
    test_data[i] = test_data_title[i] + "mid" + test_data_content[i]

In [ ]:
from transformers import CanineTokenizer, CanineForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model = CanineForSequenceClassification.from_pretrained("google/canine-c", num_labels=14).to(device)
tokenizer = CanineTokenizer.from_pretrained("google/canine-c")

In [ ]:
# Tokenize and encode the data
encoded_data = tokenizer(train_data, padding=True, truncation=True, return_tensors="pt").to(device)
test_encoded_data = tokenizer(test_data, padding=True, truncation=True, return_tensors="pt").to(device)

# Convert labels to tensors
train_labels = torch.tensor(train_labels).to(device)
test_labels = torch.tensor(test_labels).to(device)

# # Ensure your labels start from 0 for multi-class classification
# train_labels -= min(train_labels)
# test_labels -= min(test_labels)

In [ ]:
# save the tokenized data
with open('encoded_data.pkl', 'wb') as file:
    pickle.dump(encoded_data, file)

with open('test_encoded_data.pkl', 'wb') as file:
    pickle.dump(test_encoded_data, file)

# load data
with open('encoded_data.pkl', 'rb') as file:
    encoded_data = pickle.load(file)

with open('test_encoded_data.pkl', 'rb') as file:
    test_encoded_data = pickle.load(file)

In [ ]:
# print range of labels
print(min(train_labels))
print(max(train_labels))

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 4
model.train()

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(encoded_data.input_ids, encoded_data.attention_mask, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_encoded_data.input_ids, test_encoded_data.attention_mask, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    i = 0
    for batch in train_dataloader:
        input_ids, attention_mask, batch_labels = [item.to(device) for item in batch]

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if(i % 1 == 0):
            print("i: {} Loss: {:.4f}".format(batch, loss.item()))
        i += 1

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

print("Training finished")

In [ ]:
# Save your trained model in torch
model.save_pretrained("canine_dbpedia")

# Load your trained model
model = CanineForSequenceClassification.from_pretrained("canine_dbpedia")

In [ ]:
# save model in torch
torch.save(model, 'canine_dbpedia.pt')

In [ ]:
# Evaluate the model on the test data
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, batch_labels = [item.to(device) for item in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        total += batch_labels.size(0)
        correct += (predictions == batch_labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test data: {accuracy * 100:.2f}%")

In [ ]:
# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

true_labels = test_labels.tolist()
confusion = confusion_matrix(true_labels, predictions)
classification_rep = classification_report(true_labels, predictions, target_names=[str(i) for i in range(14)])

print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(classification_rep)